<a href="https://colab.research.google.com/github/DayalStrub/ecir2021tutorial/blob/main/other/2-simplest-thing-Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install -q python-terrier
!pip install -q git+https://github.com/terrier-org/pyterrier.git

In [2]:
import pandas as pd
from pandas.testing import assert_frame_equal

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize

In [3]:
import pyterrier as pt
pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


PyTerrier 0.5.0 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


In [4]:
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')

In [5]:
next(iter(dataset.get_corpus_iter()))

OrderedDict([('title',
              'Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia'),
             ('doi', '10.1186/1471-2334-1-6'),
             ('date', '2001-07-04'),
             ('abstract',
              'OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common

In [6]:
N = 10_000

pt_index_path = './index_cord19'

# create the index, using the IterDictIndexer indexer 
indexer = pt.index.IterDictIndexer(pt_index_path, overwrite=True) # , blocks=True

# we give the dataset get_corpus_iter() directly to the indexer
# while specifying the fields to index and the metadata to record
index_ref = indexer.index(
    (datum for i, datum in enumerate(dataset.get_corpus_iter()) if i < N),
    fields=('abstract',), # TODO should this be a list?
    # meta={'docno' : 26, 'text' : 2048, 'abstract' : 2048}
    meta=['docno','abstract'],
    )

index = pt.IndexFactory.of(index_ref)

09:48:12.620 [ForkJoinPool-1-worker-3] WARN  o.t.structures.indexing.Indexer - Adding an empty document to the index (6iu1dtyl) - further warnings are suppressed

09:48:24.821 [ForkJoinPool-1-worker-3] WARN  o.t.structures.indexing.Indexer - Indexed 2143 empty documents


In [7]:
print(index.getCollectionStatistics().toString())

Number of documents: 10000
Number of terms: 36191
Number of postings: 673758
Number of fields: 1
Number of tokens: 1040094
Field names: [abstract]
Positions:   false



In [8]:
queries = dataset.get_topics(variant='title').head(3)
queries

,qid,query
0,1,coronavirus origin
1,2,coronavirus response to weather changes
2,3,coronavirus immunity


In [9]:
M = 1002 # TODO is 1000 the max ever returned? Probably bug

bm25 = pt.BatchRetrieve(index, wmodel="BM25", num_results=M)
rm3 = pt.rewrite.RM3(index)
bm25_text = pt.BatchRetrieve(index, wmodel="BM25", metadata=["docno", "abstract"], num_results=M)

rm3_pipe = bm25 >> rm3 >> bm25_text >> pt.apply.rename({'abstract':'text'})

In [10]:
# TODO Question: in rm3_pipe, is final bm25 just reranking initial bm25 results, or does it re-consider all docs?

In [11]:
df_part = (bm25 >> rm3).transform(queries)
df_part

,qid,query_0,query
0,1,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
1,2,coronavirus response to weather changes,applypipeline:off respons^0.150000006 forecast...
2,3,coronavirus immunity,applypipeline:off network^0.029360194 step^0.0...


In [12]:
df_out = rm3_pipe.transform(queries)
df_out # .loc[df_out["qid"] == "1", :]

,qid,docid,docno,text,rank,score,query_0,query
0,1,2257,8y24j34j,Middle East respiratory syndrome coronavirus i...,0,19.049835,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
1,1,4582,2jq626ye,A novel coronavirus (2019-nCoV) originating in...,1,13.717366,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
2,1,1394,vnafx1ng,The phosphoprotein (P) gene of most Paramyxovi...,2,11.114468,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
3,1,1616,hp5x637c,"BACKGROUND: In 2007, a novel bunyavirus was fo...",3,8.417806,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
4,1,4936,41ui4lqc,Emerging viruses represent a continuous threat...,4,8.340423,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
...,...,...,...,...,...,...,...,...
2995,3,4382,r2s0gks8,"BACKGROUND: In endemic areas, pregnant women a...",995,2.851963,coronavirus immunity,applypipeline:off network^0.029360194 step^0.0...
2996,3,673,x5ardo5j,"In all vertebrate animals, CD8(+) cytotoxic T ...",996,2.851936,coronavirus immunity,applypipeline:off network^0.029360194 step^0.0...
2997,3,1567,y1nr8y25,OBJECTIVES: The emergence of the pandemic infl...,997,2.849012,coronavirus immunity,applypipeline:off network^0.029360194 step^0.0...
2998,3,4147,mrfvdpmk,As a response to a diverse array of external s...,998,2.848665,coronavirus immunity,applypipeline:off network^0.029360194 step^0.0...


In [13]:
df_bm = bm25.transform(queries)
df_bm

,qid,docid,docno,rank,score,query
0,1,2257,8y24j34j,0,16.990326,coronavirus origin
1,1,4582,2jq626ye,1,12.327608,coronavirus origin
2,1,1394,vnafx1ng,2,8.248003,coronavirus origin
3,1,4980,cgby8bhz,3,7.913393,coronavirus origin
4,1,5341,qbt142yj,4,7.335607,coronavirus origin
...,...,...,...,...,...,...
2348,3,4220,5ckfmq6j,995,2.448816,coronavirus immunity
2349,3,5698,1gcubssd,996,2.448816,coronavirus immunity
2350,3,1997,m31mb2kr,997,2.448816,coronavirus immunity
2351,3,4014,3gt3vlev,998,2.448816,coronavirus immunity


In [14]:
assert_frame_equal(df_bm, df_out)

AssertionError: ignored

In [15]:
df_out.loc[:, ["qid", "docid"]].groupby("qid").count()

,docid
qid,
1,1000
2,1000
3,1000


In [16]:
def pr_linear_reranker(
    df,
    *,
    n_top = 10,
    n_bottom = 100, # TODO should actually get 0 score documents? how to do this in pyterrier?
    col_text = 'text'
):
  df_tmp = df.copy()
  df_relevant = df_tmp.sort_values("rank").reset_index(drop=True).loc[0:n_top, ["docid", col_text]]
  df_relevant["label"] = 1

  df_not_relevant = df_tmp.sort_values("rank", ascending=False).reset_index(drop=True).loc[0:n_bottom, ["docid", col_text]]
  df_not_relevant["label"] = 0

  df_train = pd.concat([df_relevant, df_not_relevant])

  text_transformer = Pipeline([
      # ('bow', CountVectorizer()),
      ('tfidf', TfidfVectorizer()),
  ])

  preprocessor = ColumnTransformer(
      transformers=[
          # ('num', num_transformer, col_num),
          # ('cat', cat_transformer, col_cat),
          ('text', text_transformer, col_text)
      ]
  )

  clf = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', LogisticRegression(random_state=23))
  ])

  X_train = df_train.loc[:, ["docid", col_text]]
  y_train = df_train["label"]

  clf.fit(X_train, y_train);

  preds = clf.predict_proba(df_tmp.loc[:, ["docid", col_text]])

  df_tmp["score"] = preds[:, 1] # True class second?
  df_tmp.sort_values("score", ascending=False, inplace=True)
  # df_tmp.reset_index(drop=True, inplace=True)
  df_tmp["rank"] = [*range(df_tmp.shape[0])] # df_tmp.index
  return df_tmp

In [17]:
def normalise_score(df, norm="max"):
  normalized_data = normalize(df["score"].to_numpy().reshape(1, -1), norm=norm)[0]
  df["score"] = normalized_data
  return df

In [18]:
def full_simple(df):
    df_score = pr_linear_reranker(df)
    return normalise_score(df_score)

In [22]:
class SimpleTransformer(pt.transformer.TransformerBase):
  def transform(self, df, **kwargs):
    return df.groupby("qid").apply(full_simple).droplevel(0)

In [23]:
# df_eg = df_out.copy().loc[df_out["qid"] == "2", :].reset_index(drop=True)

# pr_linear_reranker(df_eg)

In [24]:
# df_eg["score"].to_numpy().reshape(1, -1)
# normalize(df_eg["score"].to_numpy().reshape(1, -1), norm="max")

# df_new = df_out.groupby("qid").apply(normalise_score)
# df_new.loc[df_new["score"] == 1, :]

In [25]:
qid = "0"
docno = "r9scxa76"

In [26]:
rm3_pipe = bm25 >> rm3 >> bm25_text >> pt.apply.rename({'abstract':'text'})

In [27]:
# pr_pipe = rm3_pipe >> pt.apply.by_query(full_simple)
pr_pipe = rm3_pipe >> pt.apply.by_query(pr_linear_reranker) >> pt.apply.by_query(normalise_score)

df_out_st = pr_pipe.transform(queries) # .loc[:0, :])
df_out_st # .loc[(df_out_st["qid"] == qid) & (df_out_st["docno"] == docno), :]

,qid,docid,docno,text,score,query_0,query,rank
0,1,2257,8y24j34j,Middle East respiratory syndrome coronavirus i...,1.000000,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...,0
3,1,1616,hp5x637c,"BACKGROUND: In 2007, a novel bunyavirus was fo...",0.991208,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...,1
10,1,5200,6io5yxgo,Many emerging infectious diseases in human pop...,0.981163,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...,2
7,1,3012,6eb2jgl7,"Norovirus infection, a highly prevalent condit...",0.978558,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...,3
6,1,1574,ey5xmb8a,"Passive immunity (PI), acquired through colost...",0.973537,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...,4
...,...,...,...,...,...,...,...,...
2411,3,5887,n4xv247l,Objective: This study was undertaken to examin...,0.331039,coronavirus immunity,applypipeline:off network^0.029360194 step^0.0...,995
2363,3,2797,9pmg7l7h,Background: Influenza viruses are among the ma...,0.330660,coronavirus immunity,applypipeline:off network^0.029360194 step^0.0...,996
2667,3,7394,n6utuwwt,The proportion of diarrheal illnesses of unkno...,0.329061,coronavirus immunity,applypipeline:off network^0.029360194 step^0.0...,997
2743,3,1608,vhiwrhkb,BACKGROUND: Respiratory syncytial virus (RSV) ...,0.327481,coronavirus immunity,applypipeline:off network^0.029360194 step^0.0...,998


In [28]:
len(df_out_st.index.unique())

3000

In [29]:
df_out_st.shape

(3000, 8)

In [30]:
pr_pipe_tr = rm3_pipe >> SimpleTransformer()

df_out_str = pr_pipe_tr.transform(queries) # .loc[:0, :])
df_out_str # .loc[(df_out_str["qid"] == qid) & (df_out_str["docno"] == docno), :]

,qid,docid,docno,text,rank,score,query_0,query
0,1,2257,8y24j34j,Middle East respiratory syndrome coronavirus i...,0,1.000000,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
3,1,1616,hp5x637c,"BACKGROUND: In 2007, a novel bunyavirus was fo...",1,0.991208,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
10,1,5200,6io5yxgo,Many emerging infectious diseases in human pop...,2,0.981163,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
7,1,3012,6eb2jgl7,"Norovirus infection, a highly prevalent condit...",3,0.978558,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
6,1,1574,ey5xmb8a,"Passive immunity (PI), acquired through colost...",4,0.973537,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
...,...,...,...,...,...,...,...,...
2411,3,5887,n4xv247l,Objective: This study was undertaken to examin...,995,0.331039,coronavirus immunity,applypipeline:off network^0.029360194 step^0.0...
2363,3,2797,9pmg7l7h,Background: Influenza viruses are among the ma...,996,0.330660,coronavirus immunity,applypipeline:off network^0.029360194 step^0.0...
2667,3,7394,n6utuwwt,The proportion of diarrheal illnesses of unkno...,997,0.329061,coronavirus immunity,applypipeline:off network^0.029360194 step^0.0...
2743,3,1608,vhiwrhkb,BACKGROUND: Respiratory syncytial virus (RSV) ...,998,0.327481,coronavirus immunity,applypipeline:off network^0.029360194 step^0.0...


In [31]:
df_out_str.shape

(3000, 8)

In [32]:
list(df_out_st.columns.values)

['qid', 'docid', 'docno', 'text', 'score', 'query_0', 'query', 'rank']

In [346]:
# df_out_st.merge(df_out_str, how = "inner", on=["qid", "docid", "docno", "text"]) # list(df_out_st.columns.values))

In [33]:
assert_frame_equal(df_out_st, df_out_str.loc[:, df_out_st.columns]) # .query("qid=='1'")

In [34]:
# TODO how to run rm3_pipe only once? CHECK caching

alpha = 0.5

pipe = alpha * rm3_pipe + (1 - alpha) * pr_pipe

In [36]:
# NOTE Experiment is indeed/actually a function
pt.Experiment(
    retr_systems=[bm25, rm3_pipe, pr_pipe, pipe],
    names=["BM25", "simplest_a_1", "simplest_a_0", "simplest_a_05"],
    topics=queries,
    qrels=dataset.get_qrels(), # TODO figure out cord19 qrels
    eval_metrics=["map"]
              )

,name,map
0,BM25,0.000389
1,simplest_a_1,0.000386
2,simplest_a_0,0.000928
3,simplest_a_05,0.000390
